# Introduction

1. Load in the collected data and camera parameters
2. Reformat the collected data for easier viewing and analysis
3. Calculate the projection matrices




## Load in the data

Begin by building a dictionary that consolidates all of the calibration data collected.

In [1]:

import numpy as np
import json
from pathlib import Path
import sys

for p in sys.path:
    print(p)
    
cam0_cal = Path(r"calibration_params\cam_0.json")
cam1_cal = Path(r"calibration_params\cam_1.json")
stereocam_0_1_cal = Path(r"calibration_params\stereo_cam_0_cam_1.json")

calibration_data = {
                    "cam0": cam0_cal, 
                    "cam1": cam1_cal, 
                    "stereocam": stereocam_0_1_cal
                    }


for key, json_path in calibration_data.items():
    with open(json_path) as f:
        calibration_data[key] = json.load(f)


c:\Users\Mac Prible\repos\calicam\src\triangulate\temuge_b
c:\Users\Mac Prible\anaconda3\envs\calicam\python38.zip
c:\Users\Mac Prible\anaconda3\envs\calicam\DLLs
c:\Users\Mac Prible\anaconda3\envs\calicam\lib
c:\Users\Mac Prible\anaconda3\envs\calicam

c:\Users\Mac Prible\anaconda3\envs\calicam\lib\site-packages
c:\Users\Mac Prible\anaconda3\envs\calicam\lib\site-packages\win32
c:\Users\Mac Prible\anaconda3\envs\calicam\lib\site-packages\win32\lib
c:\Users\Mac Prible\anaconda3\envs\calicam\lib\site-packages\Pythonwin


## Reconfigure the data

For ease of processing and plotting, convert the observed 2D image data into a consolidated dataframe.

In [2]:
# trying a different way to get a list of lists in the condition that I want

import pandas as pd

imgpointsA = calibration_data["stereocam"]["image_points_A"]
imgpointsB = calibration_data["stereocam"]["image_points_B"]
imgpointsID = calibration_data["stereocam"]["image_points_ID"]

frame = 0
summarized_points = []

for ID, pointA, pointB in zip(imgpointsID, imgpointsA, imgpointsB):
    for i, uv_A, uv_B in zip(ID, pointA, pointB):
        summarized_points.append([frame, int(i), uv_A[0], uv_A[1], uv_B[0], uv_B[1]])

    # print(frame)
    frame = frame + 1

df_pnts = pd.DataFrame(summarized_points,
                        columns = ["frame", "ID", "u_A", "v_A", "u_B", "v_B"])



## Define the Direct Linear Transformation to 3D Space
Isolate the parameters needed to create the projection matrices. Use them to define the Direct Linear Transformation that will be used to map the pair of 2D images to a 3D estimate. 

In [3]:
import scipy

# cam0 orientation will define the global frame of reference, so
# RT1 is this...not fully sure what this stuff means
RT1 = np.concatenate([np.eye(3), [[0],[0],[0]]], axis = -1)
mtx1 = calibration_data["cam0"]["camera_matrix"]
P1 = mtx1 @ RT1 #projection matrix for C1


R2 = calibration_data["stereocam"]["rotation_vector"]
T2 = calibration_data["stereocam"]["translation_vector"]
RT2 = np.concatenate([R2, T2], axis = -1)
mtx2 = calibration_data["cam1"]["camera_matrix"]
P2 = mtx2 @ RT2 #projection matrix for C2


# here u and v are used by convention to indicate the x,y position of the detected point in a 2D plane.
def triangulate(P1, P2, u_A, v_A, u_B, v_B):
 
    point1 = np.array([u_A, v_A])
    print(point1)
    print(P1)
    point2 = np.array([u_B, v_B])
    # print(point2)
    A = [point1[1]*P1[2,:] - P1[1,:],
         P1[0,:] - point1[0]*P1[2,:],
         point2[1]*P2[2,:] - P2[1,:],
         P2[0,:] - point2[0]*P2[2,:]
        ]
    A = np.array(A).reshape((4,4))
 
    B = A.transpose() @ A
    from scipy import linalg
    U, s, Vh = linalg.svd(B, full_matrices = False)
 
    return Vh[3,0:3]/Vh[3,3]

np.eye(3)
df_pnts
# Estimate 3D Position

## Traverse the dataframe and apply the DLT to each pair of points using the projection matrices calculated above

# see: https://stackoverflow.com/a/63832539/13563258
# this is a madlad answer right here

df_pnts[["X", "Y", "Z"]] = [triangulate(P1, P2, *a) for a in tuple(zip(df_pnts["u_A"], df_pnts["v_A"],df_pnts["u_B"],df_pnts["v_B"]))]
# df_pnts.to_csv("tracked_points.csv")


[383.86843872 654.80059814]
[[1.02172717e+03 0.00000000e+00 9.72245917e+02 0.00000000e+00]
 [0.00000000e+00 1.02115363e+03 5.89201350e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]
[469.23510742 643.61035156]
[[1.02172717e+03 0.00000000e+00 9.72245917e+02 0.00000000e+00]
 [0.00000000e+00 1.02115363e+03 5.89201350e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]
[551.41003418 633.42822266]
[[1.02172717e+03 0.00000000e+00 9.72245917e+02 0.00000000e+00]
 [0.00000000e+00 1.02115363e+03 5.89201350e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]
[397.59719849 732.44354248]
[[1.02172717e+03 0.00000000e+00 9.72245917e+02 0.00000000e+00]
 [0.00000000e+00 1.02115363e+03 5.89201350e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]
[482.16226196 720.44464111]
[[1.02172717e+03 0.00000000e+00 9.72245917e+02 0.00000000e+00]
 [0.00000000e+00 1.02115363e+03 5.892

# Inspect results for reasonability



In [8]:
import plotly.express as px


fig = px.scatter_3d(df_pnts, x='X', y='Y', z='Z',
              color='frame')
fig.show()

Things appear to be sitting in a plane nicely, even when viewed more closely for just a single frame.

In [9]:
import plotly.express as px

single_board = df_pnts[df_pnts["frame"]==3]

fig = px.scatter_3d(single_board, x='X', y='Y', z='Z', color='ID')
fig.show()

Previously I was having some issues with image distortion, but this does not appear to be the case after recalibrating. Now I will compare the distances between points in order to assess the 3d accuracy of the point estimate. 

See tutorial [here](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html) for details regarding how to undistort an image. I suspect that this may be something where it is preferable to go back to the source (i.e. perform a new calibration).

# Calculate Distance Between Calibration Corners



In [7]:
from charuco import Charuco

charuco = Charuco(4,5,11,8.5, aruco_scale=.75, square_size_overide_cm=.0525)
connected_corners = charuco.get_connected_corners()

connected_corners

adjacent_corners = [(x,y) for x,y in connected_corners if abs(x-y)==1]

def distance(ID_A, ID_B, frame):

    ptA_3d = (
    df_pnts
    .query('frame == ' + str(frame))
    .query('ID == ' + str(ID_A))
    .filter(["X", "Y", "Z"])
    ).to_numpy()

    ptB_3d = (
    df_pnts
    .query('frame == ' + str(frame))
    .query('ID == ' + str(ID_B))
    .filter(["X", "Y", "Z"])
    ).to_numpy()

    dist = np.linalg.norm(ptA_3d- ptB_3d)
    return dist

distances = []

for frame in range(0,df_pnts["frame"].max()):
    for pair in adjacent_corners:
        distances.append(distance(pair[0], pair[1], 8))

distances = pd.DataFrame(distances, columns = ["Distances"])

In [8]:
print("Mean distance between corners is ", distances.mean())
print("Standard deviation between corners is", distances.std())

Mean distance between corners is  Distances    0.054386
dtype: float64
Standard deviation between corners is Distances    0.000459
dtype: float64


OK. I'm feeling pretty good about where these calibration numbers are landing. It appears to be within 2 mm of the gold standard realitY measuring stick. An important part of any of this is going to involve some kind of visualization of markers in space, so I might as well figure out how to do that now. Also, I could fairly rapidly create an animation of the board moving through space to post to the Discord, and I feel like I might need that now.



# 3D Plotting


I originally tried working with plotly and then looked at matplotlib for 3D plotting options. Both left me with a sense that they were not the right tool for the job. I got excited by ipyvolume, but then began to have some trouble with just the basic installation of it. I will try again...


The above is not working. Go back to the tutorial example to get clarity on the format that the inputs must take.

https://ipyvolume.readthedocs.io/en/latest/examples/animation.html

It appears that it requires X,Y,Z coordinates to be converted into arrays. This will certainly be a happy thing for quickly integrating the raw output of the triangulate function, though currently presents a challenge for working with the numpy data. 

Ok. I am seeing something that looks like it might present a solution to the problem I'm dealing with:

*Can ipyvolume do a plot that update with real time data?*

https://github.com/widgetti/ipyvolume/issues/235

In [103]:
# df_ipv = df_pnts.query("frame == 4")
import ipywidgets as widgets

from time import sleep

# df_ipv = df_pnts

view_not_shown = True

for f in range(0, df_pnts["frame"].max()):

    df_ipv = df_pnts.query(f"frame == {f}")

    x = df_ipv["X"].to_numpy()
    y = df_ipv["Y"].to_numpy()
    z = df_ipv["Z"].to_numpy()
    
    if view_not_shown:
        fig = ipv.figure()
        ipv.show()
        scatter = ipv.scatter(x, y, z)
        view_not_shown = False
    else:
        # scatter.send_state('x')
        # scatter.send_state('y')
        # scatter.send_state('z')

        scatter.x = x
        scatter.y = y
        scatter.z = z

        sleep(0.1)






The above works well, but I would like to extend this to something where the frame could be advanced or decremented with a slider widget.

In [115]:
# df_ipv = df_pnts.query("frame == 4")
import ipywidgets as widgets
import ipyvolume as ipv

from time import sleep

# df_ipv = df_pnts

view_not_shown = True


df_ipv = df_pnts.query(f"frame == 1")

x = df_ipv["X"].to_numpy()
y = df_ipv["Y"].to_numpy()
z = df_ipv["Z"].to_numpy()

fig = ipv.figure()
# ipv.show()
scatter = ipv.scatter(x, y, z)


from ipywidgets import FloatSlider, ColorPicker, VBox, jslink
import cv2 as cv

size = FloatSlider(min=0, max=30, step=0.1)
size_selected = FloatSlider(min=0, max=30, step=0.1)


jslink((scatter, 'size'), (size, 'value'))
jslink((scatter, 'size_selected'), (size_selected, 'value'))

VBox([ipv.gcc(), size, size_selected])
ipv.show()
# widgets.jslink((scatter, "frame_slider"), (frame_slider, "value") )

# frame_slider = widgets.FloatSlider(min=0,max=30,step=1)

# df_ipv = df_pnts.query(f"frame == {f}")

# while True:

#     df_ipv = df_pnts.query(f"frame == {scatter.size_selected}")

#     x = df_ipv["X"].to_numpy()
#     y = df_ipv["Y"].to_numpy()
#     z = df_ipv["Z"].to_numpy()


#     scatter.x = x
#     scatter.y = y
#     scatter.z = z

    
#     if cv.waitKey(1) == ord("q"):
#         break




In [96]:

import numpy as np


# only x is a sequence of arrays
x = np.array([[-1, -0.8], [1, -0.1], [0., 0.5], [1,-1]])
y = np.array([0.0, 0.0])
z = np.array([0.0, 0.0])

color = np.array([[0,255,0], [255,0,0]])


ipv.figure()
s = ipv.scatter(x, y, z, color=color, marker='sphere', size=10)
ipv.xyzlim(-1, 1)
ipv.animation_control(s) # shows controls for animation controls
ipv.show()